In [1]:
import csv

In [2]:
import os

In [3]:
import pydicom

In [4]:
arr = []
for x in os.listdir('masks_marks'):
    if (x.find("_L_") > -1):
        fname = x.split("_")
        fname.pop(3)
        fname = "_".join(fname)
        try: 
            dcm_name = os.listdir('mri_L/{}'.format(fname))[0]
            arr.append({"csv_name": x, "folder_name": fname, "dcm_name": dcm_name})
        except:
            print("file not  found: {}".format(x))

file not  found: MR_0156_L_L5.csv
file not  found: MR_0806_L_L5.csv
file not  found: MR_0404_L_L5.csv
file not  found: MR_1019_L_T5.csv
file not  found: MR_1050_L_T5.csv
file not  found: MR_1174_L_T5.csv
file not  found: MR_0230_L_L5.csv
file not  found: MR_1077_L_L1.csv
file not  found: MR_1175_L_T4.csv
file not  found: MR_1060_L_T5.csv
file not  found: MR_0134_L_L5.csv
file not  found: MR_1084_L_L1.csv
file not  found: MR_0359_L_L5.csv
file not  found: MR_1055_L_T5.csv
file not  found: MR_1151_L_L1.csv
file not  found: MR_1070_L_T4.csv
file not  found: MR_0290_L_L5.csv
file not  found: MR_0787_L_L5.csv
file not  found: MR_0240_L_L5.csv


In [5]:
arr

[{'csv_name': 'MR_1056_L_T4.csv',
  'folder_name': 'MR_1056_L',
  'dcm_name': '00005_1.2.840.113619.2.176.3596.6906656.30257.1185792530.237.dcm'},
 {'csv_name': 'MR_1161_L_T5.csv',
  'folder_name': 'MR_1161_L',
  'dcm_name': '00007_1.2.840.113619.2.176.3596.6906656.30881.1198632244.469.dcm'},
 {'csv_name': 'MR_1184_L_T4.csv',
  'folder_name': 'MR_1184_L',
  'dcm_name': '00009_1.2.840.113619.2.176.3596.6906656.10743.1201607058.764.dcm'},
 {'csv_name': 'MR_0191_L_L5.csv',
  'folder_name': 'MR_0191_L',
  'dcm_name': '00009_1.2.840.113619.2.134.1762863790.1831.1301401312.952.dcm'},
 {'csv_name': 'MR_0822_L_L5.csv',
  'folder_name': 'MR_0822_L',
  'dcm_name': '00008_1.2.840.113619.2.176.3596.6906656.18949.1153392589.612.dcm'},
 {'csv_name': 'MR_1057_L_T5.csv',
  'folder_name': 'MR_1057_L',
  'dcm_name': '00008_1.2.840.113619.2.176.3596.6906656.7523.1186137822.495.dcm'},
 {'csv_name': 'MR_1057_L_L1.csv',
  'folder_name': 'MR_1057_L',
  'dcm_name': '00008_1.2.840.113619.2.176.3596.6906656.752

In [46]:
import matplotlib.pyplot as plt
import numpy as np
import cv2


In [57]:
exp = 20
top = Element('dataset')
name = SubElement(top, 'name')
name.text = 'Training spines'
comment = SubElement(top, 'comment')
comment.text = 'filler...'
# comment.set('test', 'hello')
images = SubElement(top, 'images')

for idx, a in enumerate(arr):
    ds = pydicom.dcmread('mri_L/{}/{}'.format(a['folder_name'], a['dcm_name']))
    xf = ds.PixelSpacing[0]
    yf = ds.PixelSpacing[1]
    px = np.array(ds.pixel_array)
    px = np.interp(px, (px.min(), px.max()), (0, 255))
    px = np.zeros(px.shape).astype(px.dtype)
    image = SubElement(images, 'image')
    image.set('file', 'dlib_masks/f_{}'.format(a['folder_name']))
    with open('masks_marks/{}'.format(a['csv_name'])) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",")
        pts = []
        for idx_2,r in enumerate(csv_reader):
            if (idx_2 > 0):
                p_x, p_y = ([int(float(x)*(1/xf)) for x in r[5:7]])
                pts.append((p_x, p_y))
            if (idx_2 % 6 == 0):
                if (len(pts) == 6):
                    mx, my, ma_x, ma_y = min(pts, key = lambda t: t[0])[0], \
                                         min(pts, key = lambda t: t[1])[1], \
                                         max(pts, key = lambda t: t[0])[0], \
                                         max(pts, key = lambda t: t[1])[1]
                    px_x = np.where(px < 9999, 0, 255)

                    cv2.fillPoly(px, [np.array(pts)], [255,255,255])
                    box = SubElement(image, 'box')
                    box.set('top', str(my))
                    box.set('left', str(mx))
                    box.set('width', str(abs(mx-ma_x)))
                    box.set('height', str(abs(my-ma_y)))
                    for i,p in enumerate(pts):
                        part = SubElement(box, 'part')
                        part.set('name', '0{}'.format(i % 6))
                        part.set('x', str(p[0]))
                        part.set('y', str(p[1]))
                pts = []
    cv2.imwrite('dlib_masks/f_{}.png'.format(a['folder_name']), px)
#         ElementTree(top).write('traininng.xml', encoding='utf8')
#         print("done with {}".format(a['folder_name']))
#     plt.imshow(px, cmap="bone")
print(prettify(top))


<?xml version="1.0" ?>
<dataset>
  <name>Training spines</name>
  <comment>filler...</comment>
  <images>
    <image file="dlib_masks/f_MR_1056_L">
      <box height="72" left="191" top="386" width="56">
        <part name="00" x="215" y="458"/>
        <part name="01" x="228" y="439"/>
        <part name="02" x="247" y="422"/>
        <part name="03" x="221" y="386"/>
        <part name="04" x="208" y="400"/>
        <part name="05" x="191" y="409"/>
      </box>
      <box height="65" left="148" top="335" width="62">
        <part name="00" x="167" y="400"/>
        <part name="01" x="188" y="387"/>
        <part name="02" x="210" y="379"/>
        <part name="03" x="199" y="335"/>
        <part name="04" x="178" y="349"/>
        <part name="05" x="148" y="360"/>
      </box>
      <box height="61" left="128" top="279" width="67">
        <part name="00" x="137" y="340"/>
        <part name="01" x="163" y="331"/>
        <part name="02" x="195" y="326"/>
        <part name="03" x="1

In [34]:
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
from xml.etree import ElementTree
from xml.dom import minidom

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

<?xml version="1.0" ?>
<dataset>
  <name>Training spines</name>
  <comment test="hello">filler...</comment>
</dataset>

